Dependencies


In [69]:
import pandas as pd
import cv2
import numpy as np
import requests
import re
import constants
import os
import requests
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
from pathlib import Path
from functools import partial
import urllib
from PIL import Image

Create the Dataset Dataframe

In [70]:
df = pd.read_csv('/home/atharva/ML challan/66e31d6ee96cd_student_resource_3/student_resource 3/dataset/train.csv')
df

,image_link,group_id,entity_name,entity_value
0,https://m.media-amazon.com/images/I/61I9XdN6OF...,748919,item_weight,500.0 gram
1,https://m.media-amazon.com/images/I/71gSRbyXmo...,916768,item_volume,1.0 cup
2,https://m.media-amazon.com/images/I/61BZ4zrjZX...,459516,item_weight,0.709 gram
3,https://m.media-amazon.com/images/I/612mrlqiI4...,459516,item_weight,0.709 gram
4,https://m.media-amazon.com/images/I/617Tl40LOX...,731432,item_weight,1400 milligram
...,...,...,...,...
263854,https://m.media-amazon.com/images/I/612J1R1xHl...,558806,height,5.0 centimetre
263855,https://m.media-amazon.com/images/I/61Blzh2+28...,470067,height,8.5 inch
263856,https://m.media-amazon.com/images/I/51MsegDL9V...,204245,height,43.2 centimetre
263857,https://m.media-amazon.com/images/I/510KhVw4VS...,752266,height,9.1 centimetre


In [71]:
df.columns

Index(['image_link', 'group_id', 'entity_name', 'entity_value'], dtype='object')

In [72]:
link, entity_name = df.loc[0, ['image_link', 'entity_name']]
print(link)
print(entity_name)

https://m.media-amazon.com/images/I/61I9XdN6OFL.jpg
item_weight


Provided Helper Functions

In [73]:
def common_mistake(unit):
    if unit in constants.allowed_units:
        return unit
    if unit.replace('ter', 'tre') in constants.allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in constants.allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in constants.allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, constants.allowed_units))
    return number, unit


def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)
    
    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)
        

Unique images by Link in Training set

In [74]:
classes= ['width','depth','height','item_weight','maximum_weight_recommendation','voltage','wattage','item_volume']
sett = set()
link_dict = {}
for j in range(len(df['image_link'])):
    link= df.loc[j,['image_link']]['image_link']
    sett.add(link)

sett

KeyboardInterrupt: 

Unique images by link in Training set : Equalising Long-Tailedness by taking more instances of classes present in lesser quantity

In [78]:
classes= {'width':50,'depth':50,'height':0,'item_weight':100,'maximum_weight_recommendation':100,'voltage':75,'wattage':75,'item_volume':75}
final_dict = {}
for key,val in classes.items():
    filter_df = df[df['entity_name'] == key]
    final_dict[key] = set()
    for j in range(3060,3160):
        link= filter_df.iloc[j]['image_link']            
        final_dict[key].add(link)

final_dict

{'width': {'https://m.media-amazon.com/images/I/31dCt+KjhRL.jpg',
  'https://m.media-amazon.com/images/I/415sV+8siEL.jpg',
  'https://m.media-amazon.com/images/I/41LOFEpRUzL.jpg',
  'https://m.media-amazon.com/images/I/41LT5OIj7mS.jpg',
  'https://m.media-amazon.com/images/I/41YGlSaiMbL.jpg',
  'https://m.media-amazon.com/images/I/41YN7hqCxKL.jpg',
  'https://m.media-amazon.com/images/I/41b6YE7VzdL.jpg',
  'https://m.media-amazon.com/images/I/41dMm2fvulL.jpg',
  'https://m.media-amazon.com/images/I/41ea-uJ5uYL.jpg',
  'https://m.media-amazon.com/images/I/41p10wkX9dL.jpg',
  'https://m.media-amazon.com/images/I/51+jfYJdOjL.jpg',
  'https://m.media-amazon.com/images/I/51-LRRN+l2L.jpg',
  'https://m.media-amazon.com/images/I/51-mYGxcCXL.jpg',
  'https://m.media-amazon.com/images/I/510clBsX3WL.jpg',
  'https://m.media-amazon.com/images/I/51200NOkFAL.jpg',
  'https://m.media-amazon.com/images/I/5146xTX43dL.jpg',
  'https://m.media-amazon.com/images/I/514C6rHxJ-L.jpg',
  'https://m.media-ama

In [57]:
len(sett)
ayudh = []
abhi = []

for i,lik in enumerate(sett):
    if i%2 != 1:
        ayudh.append(lik)
    else:
        abhi.append(lik)
    
    if len(abhi) == 1000:
        break

Creating directories for individual class instances for manual annotation

In [80]:
directory_path = '/home/atharva/ML challan/downloadV4'
for key,value in final_dict.items():
    folder_name = str(key)
    folder_path = os.path.join(directory_path, folder_name)
    os.makedirs(folder_path, exist_ok=True)
    download_images(value, folder_path)

100%|██████████| 100/100 [00:00<00:00, 101.71it/s]


In [47]:
for key,value in final_dict.items():
    print(len(os.listdir(f'/home/atharva/ML challan/downloadV2/{key}')))

50
50
0
100
100
75
75
75


IMPORTANT:
    Turning Label JSON files to txt files, in accordance to the YOLO input file formats and directory structure.

In [81]:
import json
import os

# Define the class mapping for labels to numbers (0-7)
label_to_class = {
    "width": 0,
    "depth": 1,
    "height": 2,
    "item_weight": 3,
    "maximum_weight_recommendation": 4,
    "voltage": 5,
    "wattage": 6,
    "item_volume": 7,
}

# Function to calculate the center, width, and height of the rectangle
def calculate_center_and_size(points, image_width, image_height):
    x1, y1 = points[0]
    x2, y2 = points[1]
    
    # Calculate the center of the rectangle
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    
    # Calculate the width and height of the rectangle
    width = abs(x2 - x1)
    height = abs(y2 - y1)
    
    # Normalize the values
    normalized_center_x = center_x / image_width
    normalized_center_y = center_y / image_height
    normalized_width = width / image_width
    normalized_height = height / image_height
    
    return normalized_center_x, normalized_center_y, normalized_width, normalized_height

# Function to process a single JSON file and return the formatted text output
def process_json_file(file_path):
    with open(file_path, 'r') as f:
        json_data = json.load(f)
    
    image_width = json_data["imageWidth"]
    image_height = json_data["imageHeight"]
    
    output_lines = []
    
    for shape in json_data["shapes"]:
        label = shape["label"]
        class_num = label_to_class.get(label, -1)  # Get class number, default to -1 if label is not mapped
        
        if class_num == -1:
            continue  # Skip if label not found in class mapping
        
        # Calculate normalized center, width, and height
        center_x, center_y, width, height = calculate_center_and_size(shape["points"], image_width, image_height)
        
        # Append formatted line to output
        output_lines.append(f"{class_num} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}")
    
    return "\n".join(output_lines)

# Function to process multiple files in a directory and save the output in text files
def process_directory(input_dir, output_dir):
    # Make sure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each JSON file in the directory
    count = 0
    for filename in os.listdir(input_dir):
        if filename.endswith(".json"):
            input_path = os.path.join(input_dir, filename)
            output_text = process_json_file(input_path)
            
            # Save the result to a corresponding text file
            output_filename = os.path.splitext(filename)[0] + ".txt"
            output_path = os.path.join(output_dir, output_filename)
            
            with open(output_path, 'w') as f:
                f.write(output_text)
            
            count += 1
            print(f"{count}. Processed {filename} -> {output_filename}")

# Example usage
input_directory = "/home/atharva/ML challan/jsonv4"  # Path to the folder containing the JSON files
output_directory = "/home/atharva/ML challan/labelsv4"  # Path to the folder where the text files will be saved

# Process all JSON files in the input directory and save the results
process_directory(input_directory, output_directory)


1. Processed 71qCo7RtY1L.json -> 71qCo7RtY1L.txt
2. Processed 61i81eRRNEL.json -> 61i81eRRNEL.txt
3. Processed 71XhT22P37L.json -> 71XhT22P37L.txt
4. Processed 61LqXhg1GrL.json -> 61LqXhg1GrL.txt
5. Processed 51cALX6SyQL.json -> 51cALX6SyQL.txt
6. Processed 61J8hiQaTBL.json -> 61J8hiQaTBL.txt
7. Processed 31UZRDrAQpL.json -> 31UZRDrAQpL.txt
8. Processed 71UpQ-YI5pL.json -> 71UpQ-YI5pL.txt
9. Processed 713vWl88hOL.json -> 713vWl88hOL.txt
10. Processed 71RyT8Xem2L.json -> 71RyT8Xem2L.txt
11. Processed 711hfmZHMaL.json -> 711hfmZHMaL.txt
12. Processed 519Z2Xv+dXL.json -> 519Z2Xv+dXL.txt
13. Processed 71+xVwwrrHL.json -> 71+xVwwrrHL.txt
14. Processed 71jwR5e9u+L.json -> 71jwR5e9u+L.txt
15. Processed 71Q37sqp4oL.json -> 71Q37sqp4oL.txt
16. Processed 61qXaTLY9iL.json -> 61qXaTLY9iL.txt
17. Processed 81bWmHtv+QL.json -> 81bWmHtv+QL.txt
18. Processed 81AejRuCvXL.json -> 81AejRuCvXL.txt
19. Processed 71mwmdJxOGL.json -> 71mwmdJxOGL.txt
20. Processed 71KOLrwC2wS.json -> 71KOLrwC2wS.txt
21. Proce

In [36]:
import os
import json
import shutil

# Define the paths
images_folder = "/home/atharva/ML challan/download/ayudh"
json_folder = '/home/atharva/ML challan/jsons'
destination_folder = '/home/atharva/ML challan/dataset/images'

# Create the destination folder if it doesn't exist
os.makedirs(destination_folder, exist_ok=True)

# Iterate through all files in the json_folder
for filename in os.listdir(json_folder):
    if filename.endswith('.json'):
        json_path = os.path.join(json_folder, filename)
        
        # Open and read the JSON file
        with open(json_path, 'r') as json_file:
            data = json.load(json_file)
            image_path = data.get('imagePath')
            
            if image_path:
                # Construct the full image path
                full_image_path = images_folder + "/" + image_path.split("/")[-1]
                
                if os.path.exists(full_image_path):
                    # Copy the image to the destination folder
                    shutil.copy(full_image_path, destination_folder)
                else:
                    print(f"Image not found: {full_image_path}")
            else:
                print(f"No imagePath found in {json_path}")

Image not found: /home/atharva/ML challan/download/ayudh/..\ayudh\21eGzwhsflL.jpg


In [87]:
import os
import shutil

# Paths to the folders
image_folder = '/home/atharva/ML challan/all images'
label_folder = '/home/atharva/ML challan/labelsv4'
output_folder = '/home/atharva/ML challan/present_images'

# Create the output folder if it does not exist
os.makedirs(output_folder, exist_ok=True)

# List all image files (assumes images are .jpg or .png)
image_files = [os.path.splitext(f)[0] for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

# List all .txt files
txt_files = [os.path.splitext(f)[0] for f in os.listdir(label_folder) if f.endswith('.txt')]

# Find matching image files based on the existence of .txt files
matching_files = set(image_files).intersection(txt_files)

# Copy matching image files to the output folder
for file_name in matching_files:
    for ext in ['.jpg', '.png']:  # Check both .jpg and .png extensions
        src_path = os.path.join(image_folder, file_name + ext)
        if os.path.exists(src_path):  # Ensure the image file exists before copying
            dst_path = os.path.join(output_folder, file_name + ext)
            shutil.copy(src_path, dst_path)
            print(f"Copied {src_path} to {dst_path}")

print(f"Copied {len(matching_files)} matching image files to the output folder.")


Copied /home/atharva/ML challan/all images/414H+2lx-FL.jpg to /home/atharva/ML challan/present_images/414H+2lx-FL.jpg
Copied /home/atharva/ML challan/all images/61yHG+2kG4L.jpg to /home/atharva/ML challan/present_images/61yHG+2kG4L.jpg
Copied /home/atharva/ML challan/all images/91Y6MbkP4lL.jpg to /home/atharva/ML challan/present_images/91Y6MbkP4lL.jpg
Copied /home/atharva/ML challan/all images/61Y2RtO4HqL.jpg to /home/atharva/ML challan/present_images/61Y2RtO4HqL.jpg
Copied /home/atharva/ML challan/all images/71krNThLJxL.jpg to /home/atharva/ML challan/present_images/71krNThLJxL.jpg
Copied /home/atharva/ML challan/all images/61l5Un3BBqL.jpg to /home/atharva/ML challan/present_images/61l5Un3BBqL.jpg
Copied /home/atharva/ML challan/all images/51XOIl4nH9L.jpg to /home/atharva/ML challan/present_images/51XOIl4nH9L.jpg
Copied /home/atharva/ML challan/all images/71UpQ-YI5pL.jpg to /home/atharva/ML challan/present_images/71UpQ-YI5pL.jpg
Copied /home/atharva/ML challan/all images/71Hs698ReYL.j

Conversion of combined dataset into individual class image and label folder

In [65]:
import os
import shutil

# Paths to the base image and label folders
image_folder = '/home/atharva/ML challan/dataset/train/images'
label_folder = 'dataset/train/labels'
output_base_folder = '/home/atharva/ML challan/ensemble_dataset'

# Create class-specific folders for each of the 8 classes
classes = range(8)  # Classes 0 through 7

for class_id in classes:
    os.makedirs(os.path.join(output_base_folder, f'class_{class_id}_images'), exist_ok=True)
    os.makedirs(os.path.join(output_base_folder, f'class_{class_id}_labels'), exist_ok=True)

# Helper function to get the classes in a label file
def get_classes_in_label(label_file):
    present_classes = set()
    with open(label_file, 'r') as file:
        lines = file.readlines()
        for line in lines:
            class_id = int(line.split()[0])  # Extract class ID from the line
            present_classes.add(class_id)
    return present_classes

# Process each label file and corresponding image
for label_file in os.listdir(label_folder):
    if label_file.endswith('.txt'):
        # Corresponding image file (assuming images have the same name but with .jpg or .png extension)
        image_file_jpg = os.path.splitext(label_file)[0] + '.jpg'
        image_file_png = os.path.splitext(label_file)[0] + '.png'

        # Full paths to label and image files
        label_path = os.path.join(label_folder, label_file)
        image_path_jpg = os.path.join(image_folder, image_file_jpg)
        image_path_png = os.path.join(image_folder, image_file_png)

        # Get the classes present in the label file
        present_classes = get_classes_in_label(label_path)

        # Copy the image and label to each relevant class folder
        for class_id in present_classes:
            image_output_folder = os.path.join(output_base_folder, f'class_{class_id}_images')
            label_output_folder = os.path.join(output_base_folder, f'class_{class_id}_labels')

            # Copy label file
            shutil.copy(label_path, os.path.join(label_output_folder, label_file))

            # Copy the corresponding image file (check for both .jpg and .png extensions)
            if os.path.exists(image_path_jpg):
                shutil.copy(image_path_jpg, os.path.join(image_output_folder, image_file_jpg))
            elif os.path.exists(image_path_png):
                shutil.copy(image_path_png, os.path.join(image_output_folder, image_file_png))

print("Images and labels have been separated into class-specific folders.")


Images and labels have been separated into class-specific folders.


In [89]:
import os
import shutil
import random

# Set paths
image_folder = '/home/atharva/ML challan/present_images'
label_folder = '/home/atharva/ML challan/labelsv4'

# Output paths for train and validation folders
train_image_folder = '/home/atharva/ML challan/final_dataset/train/images'
train_label_folder = '/home/atharva/ML challan/final_dataset/train/labels'
val_image_folder = '/home/atharva/ML challan/final_dataset/val/images'
val_label_folder = '/home/atharva/ML challan/final_dataset/val/labels'

# Create directories for train and validation if they don't exist
os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(train_label_folder, exist_ok=True)
os.makedirs(val_image_folder, exist_ok=True)
os.makedirs(val_label_folder, exist_ok=True)

# List all image files and corresponding label files
image_files = sorted([f for f in os.listdir(image_folder) if f.endswith('.jpg') or f.endswith('.png')])
label_files = sorted([f for f in os.listdir(label_folder) if f.endswith('.txt')])

# Ensure there are matching labels for every image
image_files = [f for f in image_files if f.replace('.jpg', '.txt').replace('.png', '.txt') in label_files]

# Shuffle the dataset
random.seed(42)  # for reproducibility
combined = list(zip(image_files, label_files))
random.shuffle(combined)
image_files, label_files = zip(*combined)

# Split based on percentage (e.g., 80% for training, 20% for validation)
split_ratio = 0.95
split_index = int(len(image_files) * split_ratio)

train_images = image_files[:split_index]
train_labels = label_files[:split_index]
val_images = image_files[split_index:]
val_labels = label_files[split_index:]

# Move files to respective directories
def move_files(file_list, src_folder, dst_folder):
    for file_name in file_list:
        src_path = os.path.join(src_folder, file_name)
        dst_path = os.path.join(dst_folder, file_name)
        shutil.copy2(src_path, dst_path)

# Move training files
move_files(train_images, image_folder, train_image_folder)
move_files(train_labels, label_folder, train_label_folder)

# Move validation files
move_files(val_images, image_folder, val_image_folder)
move_files(val_labels, label_folder, val_label_folder)

print(f"Training set: {len(train_images)} images")
print(f"Validation set: {len(val_images)} images")


Training set: 549 images
Validation set: 29 images


In [100]:
dr = pd.read_csv('/home/atharva/ML challan/66e31d6ee96cd_student_resource_3/student_resource 3/dataset/test.csv')
dr

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth
...,...,...,...,...
131182,131283,https://m.media-amazon.com/images/I/A1rVsIzEtk...,721522,maximum_weight_recommendation
131183,131284,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,item_weight
131184,131285,https://m.media-amazon.com/images/I/A1rdvZ5zDd...,603688,maximum_weight_recommendation
131185,131286,https://m.media-amazon.com/images/I/A1tnTUPyr7...,853009,item_weight


In [117]:
import glob
import pandas as pd
import re

def extract_number(filename):
    """Extracts the number from the filename."""
    match = re.search(r'(\d+)', filename)
    return int(match.group(1)) if match else float('inf')

def merge_csv_files(input_folder, output_file):
    """Merges multiple CSV files in a specified folder into a single CSV file,
    keeping the first file on top.

    Args:
        input_folder (str): The path to the folder containing the CSV files.
        output_file (str): The path and filename for the merged CSV file.
    """

    # Get a list of all CSV files in the input folder
    csv_files = glob.glob(f"{input_folder}/test_output_*.csv")

    if not csv_files:
        print("No CSV files found in the folder.")
        return

    # Sort the files numerically based on the number in their filename
    csv_files.sort(key=extract_number)

    # Read the first CSV file separately to ensure it is placed on top
    merged_df = pd.read_csv(csv_files[0])

    # Iterate through the rest of the CSV files and append them
    for file in csv_files[1:]:
        temp_df = pd.read_csv(file)
        merged_df = pd.concat([merged_df, temp_df], ignore_index=True)

    # Save the merged DataFrame to a single CSV file
    merged_df.to_csv(output_file, index=False)

    print("CSV files merged successfully!")

# Example usage:
input_folder = "/home/atharva/ML challan/final_csv"
output_file = "/home/atharva/ML challan/merged_data.csv"
merge_csv_files(input_folder, output_file)


CSV files merged successfully!


In [120]:
import pandas as pd
import re

# Load the data
lord = pd.read_csv("/home/atharva/ML challan/merged_data.csv")
lord = lord.fillna('')

test_df = pd.read_csv('/home/atharva/ML challan/66e31d6ee96cd_student_resource_3/student_resource 3/dataset/test.csv')

# Define the default units map
default_units_map = {
    "width": "centimetre",
    "depth": "centimetre",
    "height": "centimetre",
    "item_weight": "kilogram",
    "maximum_weight_recommendation": "kilogram",
    "voltage": "volt",
    "wattage": "watt",
    "item_volume": "litre"
}

# Function to add units based on the test_df and default_units_map
def add_units(row):
    entity_name = str(row['entitiy_name']).strip()  # Ensure entity_name is a string and strip whitespace
    index_value = row['index']  # Common index column
    
    # Check if entity_name is purely numeric
    if re.match(r'^\d+(\.\d+)?$', entity_name):
        # Find the row in test_df that matches the current index
        query = test_df[test_df['index'] == index_value]
        
        if not query.empty:
            # Get the relevant attribute (assumed to be 'entity_name' in test.csv) for unit mapping
            measurement_attribute = str(query.iloc[0]['entity_name']).strip().lower()  # Convert to lowercase for matching
            
            # Add the unit from default_units_map if the attribute is found
            if measurement_attribute in default_units_map:
                unit = default_units_map[measurement_attribute]
                return f"{entity_name} {unit}"  # Append the unit to the numeric value

    return entity_name  # Return the original value if no change is needed

# Apply the function to the 'entitiy_name' column
lord['entitiy_name'] = lord.apply(add_units, axis=1)

# Output result for inspection
print(lord[['index', 'entitiy_name']])


         index    entitiy_name
0            0                
1            1                
2            2                
3            3                
4            4                
...        ...             ...
119995  120079                
119996  120080  4800 milligram
119997  120081   2.02 kilogram
119998  120082                
119999  120083                

[120000 rows x 2 columns]


In [113]:
duplicates = help2[help2['index'].duplicated()]
duplicates

,index,entitiy_name
